In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import time
from collections import Counter

In [10]:
# OpenStreetMap - Public Transportation

# Define the Overpass API endpoint
overpass_url = "http://overpass-api.de/api/interpreter"

# Define the counties in the Atlanta metropolitan area
counties = [
    {"name": "Fulton County", "coordinates": "33.7490,-84.3880"},
    {"name": "DeKalb County", "coordinates": "33.7956,-84.2963"},
    {"name": "Cobb County", "coordinates": "33.9125,-84.5560"},
    {"name": "Gwinnett County", "coordinates": "33.9604,-84.0379"},
    {"name": "Clayton County", "coordinates": "33.5447,-84.3700"},
    {"name": "Cherokee County", "coordinates": "34.2514,-84.4803"},
    {"name": "Henry County", "coordinates": "33.4735,-84.0169"},
    {"name": "Douglas County", "coordinates": "33.7290,-84.7316"},
    {"name": "Fayette County", "coordinates": "33.4168,-84.4803"}
]

# "amenity"="bus_station" == "public_transport"="station" == "railway"="station"
# "public_transport"="stop_position" == "public_transport"="stop_area" == "railway"="platform" == "public_transport"="platform" == "highway"="bus_stop"

# Define a function to get public transport stops for each county
def get_transport_stops(api_url, county_name, location):
    overpass_query = f"""
    [out:json][timeout:60];
    area[name="{county_name}"]->.searchArea;
    (
      node["amenity"="bus_station"](area.searchArea);
      node["amenity"="taxi"](area.searchArea);
      node["public_transport"="station"](area.searchArea);
      node["public_transport"="stop_position"](area.searchArea);
      node["public_transport"="platform"](area.searchArea);
      node["public_transport"="stop_area"](area.searchArea);
      node["public_transport"="stop_area_group"](area.searchArea);
      node["highway"="bus_stop"](area.searchArea);
      node["highway"="platform"](area.searchArea);
      node["railway"="station"](area.searchArea);
      node["railway"="halt"](area.searchArea);
      node["railway"="platform"](area.searchArea);
      node["railway"="stop"](area.searchArea);
      node["railway"="tram_stop"](area.searchArea);
      node["railway"="subway_entrance"](area.searchArea);
    );
    out body;
    """
    
    # Request data from Overpass API
    response = requests.get(api_url, params={'data': overpass_query})
    data = response.json()

    # Extract relevant information from the response
    stops = [{
        "name": element.get("tags", {}).get("name", "Unnamed Stop"),
        "latitude": element["lat"],
        "longitude": element["lon"],
        "type": element.get("tags", {}).get("public_transport", element.get("tags", {}).get("highway", element.get("tags", {}).get("railway", "Unknown")))
    } for element in data["elements"]]

    print(f"Found {len(stops)} stops in {county_name}.")
    return stops

# Main function to fetch all stops by county
def fetch_all_stops_by_county(api_url, counties):
    all_stops = {}
    for county in counties:
        print(f"Fetching stops for {county['name']}...")
        stops = get_transport_stops(api_url, county["name"], county["coordinates"])
        all_stops[county["name"]] = stops
        time.sleep(2)  # To avoid hitting the rate limit of the Overpass API
    
    return all_stops

# Example usage
if __name__ == "__main__":
    # Fetch all stops by county
    all_stops_by_county = fetch_all_stops_by_county(overpass_url, counties)
    
    # Save the results to a JSON file
    with open("atlanta_public_transport_stops_by_county.json", "w") as json_file:
        json.dump(all_stops_by_county, json_file, indent=4)
    
    print("Data has been saved to 'atlanta_public_transport_stops_by_county.json'.")


Fetching stops for Fulton County...
Found 539 stops in Fulton County.
Fetching stops for DeKalb County...
Found 420 stops in DeKalb County.
Fetching stops for Cobb County...
Found 164 stops in Cobb County.
Fetching stops for Gwinnett County...
Found 93 stops in Gwinnett County.
Fetching stops for Clayton County...
Found 19 stops in Clayton County.
Fetching stops for Cherokee County...
Found 3 stops in Cherokee County.
Fetching stops for Henry County...
Found 14 stops in Henry County.
Fetching stops for Douglas County...
Found 1112 stops in Douglas County.
Fetching stops for Fayette County...
Found 250 stops in Fayette County.
Data has been saved to 'atlanta_public_transport_stops_by_county.json'.


In [11]:
# Define the Overpass API endpoint
overpass_url = "http://overpass-api.de/api/interpreter"

# Define the counties in the Atlanta metropolitan area
counties = [
    {"name": "Fulton County", "coordinates": "33.7490,-84.3880"},
    {"name": "DeKalb County", "coordinates": "33.7956,-84.2963"},
    {"name": "Cobb County", "coordinates": "33.9125,-84.5560"},
    {"name": "Gwinnett County", "coordinates": "33.9604,-84.0379"},
    {"name": "Clayton County", "coordinates": "33.5447,-84.3700"},
    {"name": "Cherokee County", "coordinates": "34.2514,-84.4803"},
    {"name": "Henry County", "coordinates": "33.4735,-84.0169"},
    {"name": "Douglas County", "coordinates": "33.7290,-84.7316"},
    {"name": "Fayette County", "coordinates": "33.4168,-84.4803"}
]

# Define a function to get public transport stops for each county
def get_transport_stops(api_url, county_name, location):
    overpass_query = f"""
    [out:json][timeout:60];
    area[name="{county_name}"]->.searchArea;
    (
      node["amenity"="bus_station"](area.searchArea);
      node["amenity"="taxi"](area.searchArea);
      node["public_transport"="station"](area.searchArea);
      node["public_transport"="stop_position"](area.searchArea);
      node["public_transport"="platform"](area.searchArea);
      node["public_transport"="stop_area"](area.searchArea);
      node["public_transport"="stop_area_group"](area.searchArea);
      node["highway"="bus_stop"](area.searchArea);
      node["highway"="platform"](area.searchArea);
      node["railway"="station"](area.searchArea);
      node["railway"="halt"](area.searchArea);
      node["railway"="platform"](area.searchArea);
      node["railway"="stop"](area.searchArea);
      node["railway"="tram_stop"](area.searchArea);
      node["railway"="subway_entrance"](area.searchArea);
    );
    out body;
    """
    
    # Request data from Overpass API
    response = requests.get(api_url, params={'data': overpass_query})
    data = response.json()

    # Extract relevant information and combine similar categories
    stops = [{
        "name": element.get("tags", {}).get("name", "Unnamed Stop"),
        "latitude": element["lat"],
        "longitude": element["lon"],
        "type": classify_stop_type(element.get("tags", {}))
    } for element in data["elements"]]

    print(f"Found {len(stops)} stops in {county_name}.")
    return stops

# Helper function to classify stop types into broader categories
def classify_stop_type(tags):
    # Combine station types into 'public_transport_station'
    if tags.get("amenity") == "bus_station" or tags.get("public_transport") == "station" or tags.get("railway") == "station":
        return "public_transport_station"
    
    # Combine stop types into 'public_transport_stop'
    if tags.get("public_transport") in ["stop_position", "stop_area", "platform", "stop_area_group"] or \
       tags.get("highway") == "bus_stop" or tags.get("railway") == "platform":
        return "public_transport_stop"
    
    # Default to the more general tag if none of the above match
    return tags.get("public_transport", tags.get("highway", tags.get("railway", "Unknown")))

# Main function to fetch all stops by county
def fetch_all_stops_by_county(api_url, counties):
    all_stops = {}
    for county in counties:
        print(f"Fetching stops for {county['name']}...")
        stops = get_transport_stops(api_url, county["name"], county["coordinates"])
        all_stops[county["name"]] = stops
        time.sleep(2)  # To avoid hitting the rate limit of the Overpass API
    
    return all_stops

# Example usage
if __name__ == "__main__":
    # Fetch all stops by county
    all_stops_by_county = fetch_all_stops_by_county(overpass_url, counties)
    
    # Save the results to a JSON file
    with open("atlanta_public_transport_stops_by_county.json", "w") as json_file:
        json.dump(all_stops_by_county, json_file, indent=4)
    
    print("Data has been saved to 'atlanta_public_transport_stops_by_county.json'.")


Fetching stops for Fulton County...
Found 539 stops in Fulton County.
Fetching stops for DeKalb County...
Found 420 stops in DeKalb County.
Fetching stops for Cobb County...
Found 164 stops in Cobb County.
Fetching stops for Gwinnett County...
Found 93 stops in Gwinnett County.
Fetching stops for Clayton County...
Found 19 stops in Clayton County.
Fetching stops for Cherokee County...
Found 3 stops in Cherokee County.
Fetching stops for Henry County...
Found 14 stops in Henry County.
Fetching stops for Douglas County...
Found 1112 stops in Douglas County.
Fetching stops for Fayette County...
Found 250 stops in Fayette County.
Data has been saved to 'atlanta_public_transport_stops_by_county.json'.
